# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 6 2023 12:15PM,258732,10,9233902,"Snap Medical Industries, LLC",Released
1,Mar 6 2023 12:15PM,258732,10,9234457,"Snap Medical Industries, LLC",Released
2,Mar 6 2023 12:15PM,258732,10,9234458,"Snap Medical Industries, LLC",Released
3,Mar 6 2023 12:15PM,258732,10,9234459,"Snap Medical Industries, LLC",Released
4,Mar 6 2023 12:15PM,258732,10,9234462,"Snap Medical Industries, LLC",Released
5,Mar 6 2023 12:15PM,258732,10,9234463,"Snap Medical Industries, LLC",Released
6,Mar 6 2023 12:15PM,258732,10,9234464,"Snap Medical Industries, LLC",Released
7,Mar 6 2023 12:15PM,258732,10,9234465,"Snap Medical Industries, LLC",Released
8,Mar 6 2023 12:15PM,258732,10,9234466,"Snap Medical Industries, LLC",Released
9,Mar 6 2023 12:15PM,258732,10,9234467,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,258728,Released,1
21,258729,Released,1
22,258730,Released,14
23,258731,Released,1
24,258732,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258728,NaN,NaN,1.0
258729,NaN,NaN,1.0
258730,NaN,NaN,14.0
258731,NaN,NaN,1.0
258732,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258695,1.0,8.0,20.0
258697,0.0,0.0,2.0
258704,0.0,1.0,7.0
258705,0.0,0.0,1.0
258708,0.0,13.0,17.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258695,1,8,20
258697,0,0,2
258704,0,1,7
258705,0,0,1
258708,0,13,17


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258695,1,8,20
1,258697,0,0,2
2,258704,0,1,7
3,258705,0,0,1
4,258708,0,13,17


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258695,1,8,20
1,258697,,,2
2,258704,,1,7
3,258705,,,1
4,258708,,13,17


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC"
11,Mar 6 2023 12:13PM,258731,12,"Digital Brands, LLC"
12,Mar 6 2023 12:13PM,258730,10,"Digital Brands, LLC"
26,Mar 6 2023 12:08PM,258729,10,Emerginnova
27,Mar 6 2023 12:06PM,258728,19,ACG North America LLC
28,Mar 6 2023 11:52AM,258727,12,Hush Hush
29,Mar 6 2023 11:45AM,258724,19,VITABIOTICS USA INC.
32,Mar 6 2023 11:40AM,258723,19,VITABIOTICS USA INC.
36,Mar 6 2023 11:39AM,258708,15,"Person & Covey, Inc."
66,Mar 6 2023 11:24AM,258722,20,Else Nutrition


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC",,,11
1,Mar 6 2023 12:13PM,258731,12,"Digital Brands, LLC",,,1
2,Mar 6 2023 12:13PM,258730,10,"Digital Brands, LLC",,,14
3,Mar 6 2023 12:08PM,258729,10,Emerginnova,,,1
4,Mar 6 2023 12:06PM,258728,19,ACG North America LLC,,,1
5,Mar 6 2023 11:52AM,258727,12,Hush Hush,,,1
6,Mar 6 2023 11:45AM,258724,19,VITABIOTICS USA INC.,,1,2
7,Mar 6 2023 11:40AM,258723,19,VITABIOTICS USA INC.,,2,2
8,Mar 6 2023 11:39AM,258708,15,"Person & Covey, Inc.",,13,17
9,Mar 6 2023 11:24AM,258722,20,Else Nutrition,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC",11,,
1,Mar 6 2023 12:13PM,258731,12,"Digital Brands, LLC",1,,
2,Mar 6 2023 12:13PM,258730,10,"Digital Brands, LLC",14,,
3,Mar 6 2023 12:08PM,258729,10,Emerginnova,1,,
4,Mar 6 2023 12:06PM,258728,19,ACG North America LLC,1,,
5,Mar 6 2023 11:52AM,258727,12,Hush Hush,1,,
6,Mar 6 2023 11:45AM,258724,19,VITABIOTICS USA INC.,2,1,
7,Mar 6 2023 11:40AM,258723,19,VITABIOTICS USA INC.,2,2,
8,Mar 6 2023 11:39AM,258708,15,"Person & Covey, Inc.",17,13,
9,Mar 6 2023 11:24AM,258722,20,Else Nutrition,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC",11,,
1,Mar 6 2023 12:13PM,258731,12,"Digital Brands, LLC",1,,
2,Mar 6 2023 12:13PM,258730,10,"Digital Brands, LLC",14,,
3,Mar 6 2023 12:08PM,258729,10,Emerginnova,1,,
4,Mar 6 2023 12:06PM,258728,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC",11,NaN,NaN
1,Mar 6 2023 12:13PM,258731,12,"Digital Brands, LLC",1,NaN,NaN
2,Mar 6 2023 12:13PM,258730,10,"Digital Brands, LLC",14,NaN,NaN
3,Mar 6 2023 12:08PM,258729,10,Emerginnova,1,NaN,NaN
4,Mar 6 2023 12:06PM,258728,19,ACG North America LLC,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC",11,0.0,0.0
1,Mar 6 2023 12:13PM,258731,12,"Digital Brands, LLC",1,0.0,0.0
2,Mar 6 2023 12:13PM,258730,10,"Digital Brands, LLC",14,0.0,0.0
3,Mar 6 2023 12:08PM,258729,10,Emerginnova,1,0.0,0.0
4,Mar 6 2023 12:06PM,258728,19,ACG North America LLC,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2328453,60,9.0,1.0
12,517458,2,0.0,0.0
15,258708,17,13.0,0.0
16,258705,1,0.0,0.0
19,1293588,30,3.0,0.0
20,258722,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2328453,60,9.0,1.0
1,12,517458,2,0.0,0.0
2,15,258708,17,13.0,0.0
3,16,258705,1,0.0,0.0
4,19,1293588,30,3.0,0.0
5,20,258722,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,60,9.0,1.0
1,12,2,0.0,0.0
2,15,17,13.0,0.0
3,16,1,0.0,0.0
4,19,30,3.0,0.0
5,20,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,60.0
1,12,Released,2.0
2,15,Released,17.0
3,16,Released,1.0
4,19,Released,30.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,0.0,13.0,0.0,3.0,0.0
Released,60.0,2.0,17.0,1.0,30.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,13.0,0.0,3.0,0.0
2,Released,60.0,2.0,17.0,1.0,30.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,13.0,0.0,3.0,0.0
2,Released,60.0,2.0,17.0,1.0,30.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()